In [34]:
import pandas as pd
import pickle
import random
import os
import itertools
from tqdm import tqdm
import numpy as np
from scipy.stats import pearsonr
import statsmodels.api as sm
import math
import warnings
warnings.filterwarnings("ignore")

In [26]:
def setup_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [27]:
# read datasets
corum_human = pd.read_table('../general_datasets/corum_humanComplexes.txt')
corum_human.index = corum_human['complex_id']

ge_matrix = pickle.load(open('../general_datasets/ge_matrix.pkl', 'rb'))

seeds = [0,42,100,1000,10000,100000,1000000,10000000,100000000,1000000000]

##### 1. Benchmark datasets for functional protein interactions

In [23]:
# constructing sets of intra- and inter-complex gene pairs

subunit_id = {}
for i in corum_human.index:
    subunits = tuple(sorted(list(set(corum_human['subunits_gene_name'][i].split(';')))))
    subunit_id[i] = subunits

posippis_all = set()
for i in subunit_id.keys():
    subunits = subunit_id[i]
    if len(subunits) >= 2:
        combinations = list(itertools.combinations(subunits, 2))
        for combination in combinations:
            t = sorted(list(combination))
            posippis_all.add((t[0], t[1]))

# sampling intra-complex gene pairs
posippi_seed = {}
posippi_id_seed = {}
posisubunit_id_seed = {}
for i in tqdm(subunit_id.keys()):
    subunits = subunit_id[i]
    if len(subunits) >= 2:
        combinations = sorted(list(itertools.combinations(subunits, 2)))
        if len(subunits) <= 5:
            for s in seeds:
                ppis = set()
                posisubunits = set()
                for combination in combinations:
                    t = sorted(list(combination))
                    ppis.add((t[0], t[1]))
                    posisubunits = posisubunits.union(set(combination))
                    try:
                        posippi_seed[s].add((t[0], t[1]))
                    except:
                        posippi_seed[s] = set()
                        posippi_seed[s].add((t[0], t[1]))
                try:
                    posippi_id_seed[s][i] = ppis
                except:
                    posippi_id_seed[s] = {}
                    posippi_id_seed[s][i] = ppis

                try:
                    posisubunit_id_seed[s][i] = posisubunits
                except:
                    posisubunit_id_seed[s] = {}
                    posisubunit_id_seed[s][i] = posisubunits

        else:
            for s in seeds:
                setup_seed(s)
                combinations_ = random.sample(combinations, len(subunits) * 2)
                ppis = set()
                posisubunits = set()
                for combination in combinations_:
                    t = sorted(list(combination))
                    ppis.add((t[0], t[1]))
                    posisubunits = posisubunits.union(set(combination))
                    try:
                        posippi_seed[s].add((t[0], t[1]))
                    except:
                        posippi_seed[s] = set()
                        posippi_seed[s].add((t[0], t[1]))
                try:
                    posippi_id_seed[s][i] = ppis
                except:
                    posippi_id_seed[s] = {}
                    posippi_id_seed[s][i] = ppis

                try:
                    posisubunit_id_seed[s][i] = posisubunits
                except:
                    posisubunit_id_seed[s] = {}
                    posisubunit_id_seed[s][i] = posisubunits

# sampling inter-complex gene pairs
negappi_seed = {}
for s in tqdm(seeds):
    posisubunit_id = posisubunit_id_seed[s]
    ids = sorted(list(posisubunit_id.keys()))

    negapair = sorted(list(itertools.combinations(ids, 2)))

    setup_seed(s)
    negapair = random.sample(negapair, 200000)
    negapair = sorted(list(negapair))

    negappi = set()
    for i in negapair:
        cs = sorted(list(i))
        c1, c2 = cs[0], cs[1]
        subunits_1 = sorted(list(posisubunit_id[c1]))
        subunits_2 = sorted(list(posisubunit_id[c2]))

        setup_seed(s)
        s_1 = random.choice(subunits_1)
        s_2 = random.choice(subunits_2)

        ppi = set([s_1, s_2])
        if len(ppi) == 2:
            ppi_ = sorted(list(ppi))
            negappi.add((ppi_[0], ppi_[1]))

    negappi = negappi-posippis_all
    negappi = sorted(list(negappi))

    setup_seed(s)
    random.shuffle(negappi)
    
    negappi_seed[s] = negappi

100%|██████████| 10/10 [01:28<00:00,  8.87s/it]


##### 2. Extraction of feature matrices for model training

In [ ]:
# positive examples

genes_all_key = {}
genes_all_key['CRISPR'] = list(ge_matrix['CRISPR'].columns)
genes_all_key['RNAi'] = list(ge_matrix['RNAi'].columns)

# ge_cut
ge_cut = {}
ge_cut['CRISPR'] = -0.237
ge_cut['RNAi'] = -0.325

ft_posi_seed_key = {}
for ge_key in ['CRISPR', 'RNAi']:
    ft_posi_seed = {}
    for s in seeds:
        posippi = posippi_seed[s]

        ppis_list = []
        ft_posi_list = []
        for ppi in tqdm(posippi):
            ppi_lst = sorted(list(ppi))
            g1 = ppi_lst[0]
            g2 = ppi_lst[1]

            genes_all = genes_all_key[ge_key]
            if g1 in genes_all and g2 in genes_all:
                ge_1 = ge_matrix[ge_key][g1].dropna()
                ge_2 = ge_matrix[ge_key][g2].dropna()

                ge_f_1 = ge_1[ge_1 < ge_cut[ge_key]]
                ge_f_2 = ge_2[ge_2 < ge_cut[ge_key]]

                ccls_inter = list(set(ge_f_1.index) & set(ge_f_2.index))
                ge_final_1 = ge_f_1[ccls_inter]
                ge_final_2 = ge_f_2[ccls_inter]

                if len(ccls_inter) >= 10:
                    # pearson相关性系数
                    pearson_r = pearsonr(ge_final_1, ge_final_2)

                    pearson_coeff = pearson_r[0]
                    pearson_p = pearson_r[1]

                    # ols模型
                    X_with_intercept = sm.add_constant(ge_final_1)
                    model_ols = sm.OLS(ge_final_2, X_with_intercept)
                    ols_r = model_ols.fit()

                    ols_coeff_0 = ols_r.params[0]
                    ols_se_0 = ols_r.bse[0]

                    ols_coeff_1 = ols_r.params[1]
                    ols_se_1 = ols_r.bse[1]

                    g1_essen = math.sqrt(
                        np.sum([np.power((i - 0), 2) for i in ge_final_1]))
                    g2_essen = math.sqrt(
                        np.sum([np.power((i - 0), 2) for i in ge_final_2]))

                    g1_mean = np.mean(ge_final_1)
                    g2_mean = np.mean(ge_final_2)
                    g1_std = np.std(ge_final_1)
                    g2_std = np.std(ge_final_2)

                    g1_min = min(ge_final_1)
                    g1_25 = np.percentile(ge_final_1, 25)
                    g1_50 = np.percentile(ge_final_1, 50)
                    g1_75 = np.percentile(ge_final_1, 75)
                    g1_max = max(ge_final_1)

                    g2_min = min(ge_final_2)
                    g2_25 = np.percentile(ge_final_2, 25)
                    g2_50 = np.percentile(ge_final_2, 50)
                    g2_75 = np.percentile(ge_final_2, 75)
                    g2_max = max(ge_final_2)

                    ge_e_1 = []
                    for k in ge_1:
                        if k > 0:
                            ge_e_1.append(0)
                        else:
                            ge_e_1.append(k)

                    ge_e_2 = []
                    for k in ge_2:
                        if k > 0:
                            ge_e_2.append(0)
                        else:
                            ge_e_2.append(k)

                    g1_essen_all = math.sqrt(
                        np.sum([np.power((i - 0), 2) for i in ge_e_1]))
                    g2_essen_all = math.sqrt(
                        np.sum([np.power((i - 0), 2) for i in ge_e_2]))

                    g1_mean_all = np.mean(ge_e_1)
                    g2_mean_all = np.mean(ge_e_2)
                    g1_std_all = np.std(ge_e_1)
                    g2_std_all = np.std(ge_e_2)

                    g1_min_all = min(ge_e_1)
                    g1_25_all = np.percentile(ge_e_1, 25)
                    g1_50_all = np.percentile(ge_e_1, 50)
                    g1_75_all = np.percentile(ge_e_1, 75)
                    g1_max_all = max(ge_e_1)

                    g2_min_all = min(ge_e_2)
                    g2_25_all = np.percentile(ge_e_2, 25)
                    g2_50_all = np.percentile(ge_e_2, 50)
                    g2_75_all = np.percentile(ge_e_2, 75)
                    g2_max_all = max(ge_e_2)

                    features = [len(ccls_inter), pearson_coeff, pearson_p,
                                ols_coeff_0, ols_se_0,
                                ols_coeff_1, ols_se_1,
                                g1_essen, g2_essen,
                                g1_essen_all, g2_essen_all,
                                g1_mean, g2_mean, g1_std, g2_std,
                                g1_25, g1_50, g1_75, g2_25, g2_50, g2_75,
                                g1_min, g2_min, g1_max, g2_max,
                                g1_mean_all, g2_mean_all, g1_std_all, g2_std_all,
                                g1_25_all, g1_50_all, g1_75_all, g2_25_all, g2_50_all, g2_75_all,
                                g1_min_all, g2_min_all, g1_max_all, g2_max_all,
                                ]
                    ppis_list.append(ppi)
                    ft_posi_list.append(features)

        ft_posi_seed[s] = pd.DataFrame(ft_posi_list, index=ppis_list,
                                       columns=['SCCLs_number', 'pearson_coeff', 'pearson_p',
                                                'ols_coeff_0', 'ols_se_0',
                                                'ols_coeff_1', 'ols_se_1',
                                                'g1_essen', 'g2_essen',
                                                'g1_essen_all', 'g2_essen_all',
                                                'g1_mean', 'g2_mean', 'g1_std', 'g2_std',
                                                'g1_25', 'g1_50', 'g1_75', 'g2_25', 'g2_50', 'g2_75',
                                                'g1_min', 'g2_min', 'g1_max', 'g2_max',
                                                'g1_mean_all', 'g2_mean_all', 'g1_std_all', 'g2_std_all',
                                                'g1_25_all', 'g1_50_all', 'g1_75_all', 'g2_25_all', 'g2_50_all',
                                                'g2_75_all',
                                                'g1_min_all', 'g2_min_all', 'g1_max_all', 'g2_max_all',
                                                ])
    ft_posi_seed_key[ge_key] = ft_posi_seed

In [ ]:
# negative examples

genes_all_key = {}
genes_all_key['CRISPR'] = list(ge_matrix['CRISPR'].columns)
genes_all_key['RNAi'] = list(ge_matrix['RNAi'].columns)

# ge_cut
ge_cut = {}
ge_cut['CRISPR'] = -0.237
ge_cut['RNAi'] = -0.325

ft_nega_seed_key = {}
for ge_key in ['CRISPR', 'RNAi']:
    ft_posi_seed = ft_posi_seed_key[ge_key]
    
    ft_nega_seed = {}
    for s in seeds:
        ft_posi = ft_posi_seed[s]

        negappi = negappi_seed[s]

        ppis_list = []
        ft_nega_list = []
        for ppi in tqdm(negappi):
            ppi_lst = sorted(list(ppi))
            g1 = ppi_lst[0]
            g2 = ppi_lst[1]

            genes_all = genes_all_key[ge_key]
            if g1 in genes_all and g2 in genes_all:
                ge_1 = ge_matrix_dict[ge_key][g1].dropna()
                ge_2 = ge_matrix_dict[ge_key][g2].dropna()

                ge_f_1 = ge_1[ge_1 < ge_cut[ge_key]]
                ge_f_2 = ge_2[ge_2 < ge_cut[ge_key]]

                ccls_inter = list(set(ge_f_1.index) & set(ge_f_2.index))
                ge_final_1 = ge_f_1[ccls_inter]
                ge_final_2 = ge_f_2[ccls_inter]

                if len(ccls_inter) >= 10:
                    # pearson相关性系数
                    pearson_r = pearsonr(ge_final_1, ge_final_2)

                    pearson_coeff = pearson_r[0]
                    pearson_p = pearson_r[1]

                    # ols模型
                    X_with_intercept = sm.add_constant(ge_final_1)
                    model_ols = sm.OLS(ge_final_2, X_with_intercept)
                    ols_r = model_ols.fit()

                    ols_coeff_0 = ols_r.params[0]
                    ols_se_0 = ols_r.bse[0]

                    ols_coeff_1 = ols_r.params[1]
                    ols_se_1 = ols_r.bse[1]

                    g1_essen = math.sqrt(
                        np.sum([np.power((i - 0), 2) for i in ge_final_1]))
                    g2_essen = math.sqrt(
                        np.sum([np.power((i - 0), 2) for i in ge_final_2]))

                    g1_mean = np.mean(ge_final_1)
                    g2_mean = np.mean(ge_final_2)
                    g1_std = np.std(ge_final_1)
                    g2_std = np.std(ge_final_2)

                    g1_min = min(ge_final_1)
                    g1_25 = np.percentile(ge_final_1, 25)
                    g1_50 = np.percentile(ge_final_1, 50)
                    g1_75 = np.percentile(ge_final_1, 75)
                    g1_max = max(ge_final_1)

                    g2_min = min(ge_final_2)
                    g2_25 = np.percentile(ge_final_2, 25)
                    g2_50 = np.percentile(ge_final_2, 50)
                    g2_75 = np.percentile(ge_final_2, 75)
                    g2_max = max(ge_final_2)

                    ge_e_1 = []
                    for k in ge_1:
                        if k > 0:
                            ge_e_1.append(0)
                        else:
                            ge_e_1.append(k)

                    ge_e_2 = []
                    for k in ge_2:
                        if k > 0:
                            ge_e_2.append(0)
                        else:
                            ge_e_2.append(k)

                    g1_essen_all = math.sqrt(
                        np.sum([np.power((i - 0), 2) for i in ge_e_1]))
                    g2_essen_all = math.sqrt(
                        np.sum([np.power((i - 0), 2) for i in ge_e_2]))

                    g1_mean_all = np.mean(ge_e_1)
                    g2_mean_all = np.mean(ge_e_2)
                    g1_std_all = np.std(ge_e_1)
                    g2_std_all = np.std(ge_e_2)

                    g1_min_all = min(ge_e_1)
                    g1_25_all = np.percentile(ge_e_1, 25)
                    g1_50_all = np.percentile(ge_e_1, 50)
                    g1_75_all = np.percentile(ge_e_1, 75)
                    g1_max_all = max(ge_e_1)

                    g2_min_all = min(ge_e_2)
                    g2_25_all = np.percentile(ge_e_2, 25)
                    g2_50_all = np.percentile(ge_e_2, 50)
                    g2_75_all = np.percentile(ge_e_2, 75)
                    g2_max_all = max(ge_e_2)

                    features = [len(ccls_inter), pearson_coeff, pearson_p,
                                ols_coeff_0, ols_se_0,
                                ols_coeff_1, ols_se_1,
                                g1_essen, g2_essen,
                                g1_essen_all, g2_essen_all,
                                g1_mean, g2_mean, g1_std, g2_std,
                                g1_25, g1_50, g1_75, g2_25, g2_50, g2_75,
                                g1_min, g2_min, g1_max, g2_max,
                                g1_mean_all, g2_mean_all, g1_std_all, g2_std_all,
                                g1_25_all, g1_50_all, g1_75_all, g2_25_all, g2_50_all, g2_75_all,
                                g1_min_all, g2_min_all, g1_max_all, g2_max_all,
                                ]
                    ppis_list.append(ppi)
                    ft_nega_list.append(features)

                    if len(ft_nega_list) == len(ft_posi):
                        print(len(ft_nega_list))
                        break

        ft_nega_seed[s] = pd.DataFrame(ft_nega_list, index=ppis_list,
                                       columns=['SCCLs_number', 'pearson_coeff', 'pearson_p',
                                      'ols_coeff_0', 'ols_se_0',
                                      'ols_coeff_1', 'ols_se_1',
                                      'g1_essen', 'g2_essen',
                                      'g1_essen_all', 'g2_essen_all',
                                      'g1_mean', 'g2_mean', 'g1_std', 'g2_std',
                                      'g1_25', 'g1_50', 'g1_75', 'g2_25', 'g2_50', 'g2_75',
                                      'g1_min', 'g2_min', 'g1_max', 'g2_max',
                                      'g1_mean_all', 'g2_mean_all', 'g1_std_all', 'g2_std_all',
                                      'g1_25_all', 'g1_50_all', 'g1_75_all', 'g2_25_all', 'g2_50_all',
                                      'g2_75_all',
                                      'g1_min_all', 'g2_min_all', 'g1_max_all', 'g2_max_all',
                                      ])
    ft_nega_seed_key[ge_key] = ft_nega_seed